In [1]:
'''
TODO:
- Transform suspicious text to embeddings
- Similar Documents Retrieval: get top 5 similar documents
- Detailed Analysis: get sentences pairs that have high cosine similarity
'''
from get_embeddings import mean_pooling_bert
from similar_documents_retrieval import get_similar_docs_by_cos_sims, get_similar_docs_by_cos_sims_without_clustering
from detailed_analysis import get_plagiarised_pairs
from datetime import datetime, timedelta
from nltk.tokenize import sent_tokenize
from pickle import load, dump

with open('../data/indosum/df.pkl', 'rb') as f:
    corpus = load(f)

In [4]:
def transform_suspicious_text(sus_text):
    print('[INFO] Transforming suspicious text into IndoBERT embeddings...')
    embeddings = mean_pooling_bert(text=sus_text)
    return embeddings

def get_n_similar_documents(sus_text, n=5, clustering=True):
    embeddings = transform_suspicious_text(sus_text=sus_text)
    print(f'[INFO] Getting {n} similar documents by cosine similarity...')
    if clustering == True:
        similar_docs = get_similar_docs_by_cos_sims(sus_doc_embeddings=embeddings, n=n)
    else:
        similar_docs = get_similar_docs_by_cos_sims_without_clustering(sus_doc_embeddings=embeddings, n=n)

    return similar_docs

In [2]:

def detailed_plagiarism(sus_text, n=5, threshold=.9, clustering=True):
    sent_tok = sent_tokenize(sus_text)
    similar_sources = get_n_similar_documents(sus_text=sus_text, n=n, clustering=clustering)
    similar_source_docs = {}
    suspected_plagiarism_cases = {}
    print('[INFO] Getting source documents..')
    for src_idx in similar_sources:
        similar_source_docs[src_idx] = corpus['paragraphs_sent_tok'][src_idx]
        print(f'[INFO] Source document {src_idx} appended')

    print('[INFO] Performing detailed analysis...')
    list_of_source_doc_urls = []
    suspected_plagiarised_sentences = []
    sum_cos_sim = 0
    for doc_idx, doc in similar_source_docs.items():
        # print(doc)
        print(f'[INFO] Checking source document {doc_idx}')
        src_document_url = corpus['source_url'][src_idx]
        if src_document_url not in list_of_source_doc_urls:
            list_of_source_doc_urls.append(src_document_url)
            plagiarised_pairs = get_plagiarised_pairs(src_doc=doc, sus_doc=sus_text, threshold=threshold)
            for pair in plagiarised_pairs:
                input_sentence = pair['input_sentence']
                if input_sentence not in suspected_plagiarised_sentences:
                    suspected_plagiarised_sentences.append({input_sentence: pair['similarity_score']})
                    sum_cos_sim += pair['similarity_score']
                else:
                    idx_to_be_deleted = 0
                    for idx, item in enumerate(suspected_plagiarised_sentences):
                        if pair['similarity_score'] > item[input_sentence]:
                            idx_to_be_deleted = idx
                            del suspected_plagiarised_sentences[idx_to_be_deleted]
                            suspected_plagiarised_sentences.append({input_sentence: pair['similarity_score']})
                            sum_cos_sim += pair['similarity_score']

            suspected_plagiarism_cases[doc_idx] = {
                'source_doc_url': corpus['source_url'][src_idx],
                'plagiarised_pairs': plagiarised_pairs
            }
        print(f'[INFO] Source document {doc_idx} checked')

    return {'overall_plagiarism_score': sum_cos_sim/len(sent_tok), 'suspected_sentences': suspected_plagiarism_cases}


In [5]:
document = '''Versi web Google Calendar memiliki sesuatu yang baru untuk ditawarkan. Penampilannya telah sepenuhnya diperbarui, mengadopsi gaya desain modern yang sebelumnya tersedia pada aplikasi Google Calendar di perangkat mobile. Secara keseluruhan, versi Calendar baru terlihat lebih rapi dan lebih mudah untuk diarahkan. Layout responsif yang digunakan berarti penampilannya akan disesuaikan secara otomatis dengan ukuran layar dan jendela browser. Bukan hanya berubah secara visual, versi Calendar baru juga membawa beberapa fitur yang berguna. Salah satunya adalah kemampuan untuk memformat dan menambahkan tautan pada undangan, sehingga pengguna dapat membuat jadwal yang lebih detail dan memberikan akses kepada materi yang dibutuhkan oleh peserta lain sebelum rapat dimulai. Calendar sekarang juga membuat lebih mudah untuk melihat dan mengelola beberapa kalender dari akun yang berbeda dalam tampilan harian. Fitur ini tentunya akan sangat membantu pekerja yang harus menjadwalkan rapat untuk anggota tim mereka. Terakhir, saat membuat jadwal rapat baru, pengguna dapat menambahkan informasi yang lebih detail tentang ruang rapat, termasuk lokasi, kapasitas, peralatan audio dan video, dan informasi tentang aksesibilitas bagi peserta yang berkedudukan kursi roda. Bagi yang menggunakan Calendar untuk akun pribadi mereka, mereka bisa mengaktifkan tampilan baru dengan mengklik tombol "Use new Calendar" di ujung kanan atas. Administrator G Suite juga dapat mengaktifkan tampilan baru Calendar mulai hari ini. Sumber: Google. DailySocial.id adalah portal berita startup dan inovasi teknologi. Kamu bisa menjadi anggota komunitas startup dan inovasi DailySocial.id, mengunduh laporan riset dan statistik seputar teknologi secara gratis, dan mengikuti berita startup dan gadget terbaru di Indonesia.'''
similar_docs = get_n_similar_documents(sus_text=document, n=10, clustering=False)
similar_docs

[INFO] Transforming suspicious text into IndoBERT embeddings...
[INFO] Getting 10 similar documents by cosine similarity...


100%|██████████| 93870/93870 [00:11<00:00, 8339.64it/s]


OrderedDict([(5, 0.9243268370628357),
             (22493, 0.9243268370628357),
             (41106, 0.9243268370628357),
             (60767, 0.9243268370628357),
             (75240, 0.9243268370628357),
             (21905, 0.8875881433486938),
             (28009, 0.8875881433486938),
             (47722, 0.8875881433486938),
             (53338, 0.8875881433486938),
             (76126, 0.8875881433486938)])

In [6]:
document = '''Versi web Google Calendar memiliki sesuatu yang baru untuk ditawarkan. Penampilannya telah sepenuhnya diperbarui, mengadopsi gaya desain modern yang sebelumnya tersedia pada aplikasi Google Calendar di perangkat mobile. Secara keseluruhan, versi Calendar baru terlihat lebih rapi dan lebih mudah untuk diarahkan. Layout responsif yang digunakan berarti penampilannya akan disesuaikan secara otomatis dengan ukuran layar dan jendela browser. Bukan hanya berubah secara visual, versi Calendar baru juga membawa beberapa fitur yang berguna. Salah satunya adalah kemampuan untuk memformat dan menambahkan tautan pada undangan, sehingga pengguna dapat membuat jadwal yang lebih detail dan memberikan akses kepada materi yang dibutuhkan oleh peserta lain sebelum rapat dimulai. Calendar sekarang juga membuat lebih mudah untuk melihat dan mengelola beberapa kalender dari akun yang berbeda dalam tampilan harian. Fitur ini tentunya akan sangat membantu pekerja yang harus menjadwalkan rapat untuk anggota tim mereka. Terakhir, saat membuat jadwal rapat baru, pengguna dapat menambahkan informasi yang lebih detail tentang ruang rapat, termasuk lokasi, kapasitas, peralatan audio dan video, dan informasi tentang aksesibilitas bagi peserta yang berkedudukan kursi roda. Bagi yang menggunakan Calendar untuk akun pribadi mereka, mereka bisa mengaktifkan tampilan baru dengan mengklik tombol "Use new Calendar" di ujung kanan atas. Administrator G Suite juga dapat mengaktifkan tampilan baru Calendar mulai hari ini. Sumber: Google. DailySocial.id adalah portal berita startup dan inovasi teknologi. Kamu bisa menjadi anggota komunitas startup dan inovasi DailySocial.id, mengunduh laporan riset dan statistik seputar teknologi secara gratis, dan mengikuti berita startup dan gadget terbaru di Indonesia.'''
similar_docs = get_n_similar_documents(sus_text=document, n=10, clustering=False)
similar_docs

[INFO] Transforming suspicious text into IndoBERT embeddings...
[INFO] Getting 10 similar documents by cosine similarity...


100%|██████████| 93870/93870 [00:10<00:00, 8880.29it/s]


OrderedDict([(5, 0.9243268370628357),
             (22493, 0.9243268370628357),
             (41106, 0.9243268370628357),
             (60767, 0.9243268370628357),
             (75240, 0.9243268370628357),
             (21905, 0.8875881433486938),
             (28009, 0.8875881433486938),
             (47722, 0.8875881433486938),
             (53338, 0.8875881433486938),
             (76126, 0.8875881433486938)])

In [7]:
document = '''Timnas Indonesia kembali diturunkan setelah pelatih Alfred Riedl memutuskan melepas Rizuki Pera, Septian David Moran, dan Do Mingus Fakdaba. Tiga pemain tersisih setelah timnas Indonesia kembali dari uji coba di Myanmar dan Vietnam. Rizky, David dan Fakdaver jelas tak akan mengikuti pemusatan latihan di Karavaci, Tangerang. “Setelah kembali dari Vietnam, kami melepas Septian David, Lizky Pera yang masuk daftar tunggu, dan Do Mingus.-Catatan: Bagaimana Riedl membentuk Timnas Indonesia Ahli taktik asal Austria itu tidak memberikan detail apapun tentang tindakan disipliner apa yang diambil Fakdaver. Ini sesi latihan nasional di Piala AFF 2016. Riedl memanggil penjaga gawang Semyon Padang Jandia Ekputra, yang sebelumnya dipertahankan untuk pertandingan Tes di Myanmar dan Vietnam.'''
similar_docs2 = get_n_similar_documents(sus_text=document, n=10, clustering=True)
similar_docs2

[INFO] Transforming suspicious text into IndoBERT embeddings...
[INFO] Getting 10 similar documents by cosine similarity...


OrderedDict([(11613, 0.9105722904205322),
             (31487, 0.9105722904205322),
             (44333, 0.9105722904205322),
             (68629, 0.9105722904205322),
             (89641, 0.9105722904205322),
             (660, 0.8248763084411621),
             (2972, 0.8248763084411621),
             (20834, 0.8248763084411621),
             (39371, 0.8248763084411621),
             (57621, 0.8248763084411621)])

In [8]:
document = '''Meksiko menunjukkan ketahanan lagi. Kamis (22/06), tertinggal dari El Tori berhasil membalikkan keadaan dan akhirnya menjadi 2 - 1 di leg kedua Grup A Piala Konfederasi 2017 di Stadion Olimpiade Fisht Sochi melawan Selandia Baru. Namun, dua gol Raul Jimenez (54') dan Oribe Peralta (72') akhirnya memupus harapan All Whites untuk bangkit di turnamen tersebut. Tambahan tiga poin itu untuk sementara membawa Meksiko ke puncak klasemen Grup A dengan raihan empat poin. Poin mereka sebenarnya sama dengan Portugal, namun tim asuhan Juan Carlos Osorio memiliki kemampuan mencetak gol yang lebih baik. Selandia Baru, sebaliknya, belum mencetak satu poin pun dalam dua pertandingan, sehingga dipastikan akan tersingkir. Meksiko tidak menunjukkan sisi terbaiknya kali ini. Osorio melakukan delapan pergantian pemain dari hasil imbang melawan Portugal. Pemain seperti Marco Fabian, Diego Reyes, Giovani dos Santos dan Oribe Peralta mendapat kesempatan bermain sejak awal. Tak hanya itu, formasi dasar kali ini diubah menjadi 3-4-3, namun sebenarnya perubahan tersebut tidak berdampak besar pada keseluruhan pertandingan melawan Meksiko. Bukti bahwa mereka masih bisa mendominasi lawan, baik dalam penguasaan bola maupun dalam menciptakan peluang. Secara keseluruhan, Meksiko menguasai bola 65% dari waktu. Selain itu, mereka mampu menciptakan 16 peluang mencetak gol, 10 di antaranya tepat sasaran. Tapi mungkin karakter Meksiko tertinggal lebih dulu. Pasalnya, meski selalu dikunci, Selandia Baru justru lebih dulu tampil baik berkat Wood. Pencetak gol terbanyak divisi Championship di Leeds United itu memanfaatkan kekeliruan pertahanan Meksiko, yang mampu menaklukkan kiper Alfredo Tavarel. Begitu babak kedua dimulai, Meksiko mulai. Hanya sembilan menit setelah kick-off, Jiménez berhasil mengakhiri serangan lima pemain dengan tembakan keras dari area penalti. Gol kemenangan Meksiko dicetak pada menit ke-72 setelah tembakan brilian dari kiri oleh Javier Aquino dilanjutkan dengan jebakan yang dengan mudah dikonversi Peralta menjadi gol. Patut dicatat bahwa dalam pertandingan melawan Portugal, Meksiko tertinggal dua kali sebelum akhirnya merebut poin. Di laga ini juga Meksiko harus kebobolan terlebih dahulu untuk akhirnya mencetak gol dan menang. Selain tayangan ulang tersebut, pertandingan tersebut memiliki tiga acara menarik. Pertama, cederanya bek utama Carlos Salcedo. Pemain Eintracht Frankfurt itu diganti pada babak pertama karena cedera bahu. Kedua, pertarungan antara pemain Meksiko dan Selandia Baru setelah pertarungan sengit antara Michael Boxall dan Hector Herrera Terakhir, pidato khusus oleh Rafael Marquez. Marquez yang diperkuat Barcelona digantikan oleh Osorio dan Hector Moreno pada menit ke-68. Moreno sendiri merupakan pemain yang menggantikan Salcedo di babak pertama. Dengan menghadapi Marquez, pemain berusia 38 tahun itu resmi menjadi pemain tertua kedua dalam sejarah Piala Konfederasi. Di pertandingan final, Meksiko akan menghadapi Rusia. Hasil imbang sudah cukup untuk melaju ke semifinal El Tri, terlepas dari hasil yang didapat Portugal dalam pertandingan melawan Selandia Baru. Namun, jika ingin menghindari juara Grup B, tujuan utama Anda adalah menang.'''
similar_docs3 = get_n_similar_documents(sus_text=document, n=10, clustering=True)
similar_docs3

[INFO] Transforming suspicious text into IndoBERT embeddings...
[INFO] Getting 10 similar documents by cosine similarity...


OrderedDict([(8268, 0.9727922677993774),
             (32163, 0.9727922677993774),
             (58891, 0.9727922677993774),
             (65734, 0.9727922677993774),
             (91936, 0.9727922677993774),
             (18609, 0.9379740357398987),
             (29988, 0.9379740357398987),
             (47549, 0.9379740357398987),
             (52558, 0.9379740357398987),
             (84680, 0.9379740357398987)])

In [11]:
from tqdm import tqdm

In [29]:
a = '''Versi web Google Calendar memiliki sesuatu yang baru untuk ditawarkan. Penampilannya telah sepenuhnya diperbarui, mengadopsi gaya desain modern yang sebelumnya tersedia pada aplikasi Google Calendar di perangkat mobile. Secara keseluruhan, versi Calendar baru terlihat lebih rapi dan lebih mudah untuk diarahkan. Layout responsif yang digunakan berarti penampilannya akan disesuaikan secara otomatis dengan ukuran layar dan jendela browser. Bukan hanya berubah secara visual, versi Calendar baru juga membawa beberapa fitur yang berguna. Salah satunya adalah kemampuan untuk memformat dan menambahkan tautan pada undangan, sehingga pengguna dapat membuat jadwal yang lebih detail dan memberikan akses kepada materi yang dibutuhkan oleh peserta lain sebelum rapat dimulai. Calendar sekarang juga membuat lebih mudah untuk melihat dan mengelola beberapa kalender dari akun yang berbeda dalam tampilan harian. Fitur ini tentunya akan sangat membantu pekerja yang harus menjadwalkan rapat untuk anggota tim mereka. Terakhir, saat membuat jadwal rapat baru, pengguna dapat menambahkan informasi yang lebih detail tentang ruang rapat, termasuk lokasi, kapasitas, peralatan audio dan video, dan informasi tentang aksesibilitas bagi peserta yang berkedudukan kursi roda. Bagi yang menggunakan Calendar untuk akun pribadi mereka, mereka bisa mengaktifkan tampilan baru dengan mengklik tombol "Use new Calendar" di ujung kanan atas. Administrator G Suite juga dapat mengaktifkan tampilan baru Calendar mulai hari ini. Sumber: Google. DailySocial.id adalah portal berita startup dan inovasi teknologi. Kamu bisa menjadi anggota komunitas startup dan inovasi DailySocial.id, mengunduh laporan riset dan statistik seputar teknologi secara gratis, dan mengikuti berita startup dan gadget terbaru di Indonesia.'''
b = '''Meksiko menunjukkan ketahanan lagi. Kamis (22/06), tertinggal dari El Tori berhasil membalikkan keadaan dan akhirnya menjadi 2 - 1 di leg kedua Grup A Piala Konfederasi 2017 di Stadion Olimpiade Fisht Sochi melawan Selandia Baru. Namun, dua gol Raul Jimenez (54') dan Oribe Peralta (72') akhirnya memupus harapan All Whites untuk bangkit di turnamen tersebut. Tambahan tiga poin itu untuk sementara membawa Meksiko ke puncak klasemen Grup A dengan raihan empat poin. Poin mereka sebenarnya sama dengan Portugal, namun tim asuhan Juan Carlos Osorio memiliki kemampuan mencetak gol yang lebih baik. Selandia Baru, sebaliknya, belum mencetak satu poin pun dalam dua pertandingan, sehingga dipastikan akan tersingkir. Meksiko tidak menunjukkan sisi terbaiknya kali ini. Osorio melakukan delapan pergantian pemain dari hasil imbang melawan Portugal. Pemain seperti Marco Fabian, Diego Reyes, Giovani dos Santos dan Oribe Peralta mendapat kesempatan bermain sejak awal. Tak hanya itu, formasi dasar kali ini diubah menjadi 3-4-3, namun sebenarnya perubahan tersebut tidak berdampak besar pada keseluruhan pertandingan melawan Meksiko. Bukti bahwa mereka masih bisa mendominasi lawan, baik dalam penguasaan bola maupun dalam menciptakan peluang. Secara keseluruhan, Meksiko menguasai bola 65% dari waktu. Selain itu, mereka mampu menciptakan 16 peluang mencetak gol, 10 di antaranya tepat sasaran. Tapi mungkin karakter Meksiko tertinggal lebih dulu. Pasalnya, meski selalu dikunci, Selandia Baru justru lebih dulu tampil baik berkat Wood. Pencetak gol terbanyak divisi Championship di Leeds United itu memanfaatkan kekeliruan pertahanan Meksiko, yang mampu menaklukkan kiper Alfredo Tavarel. Begitu babak kedua dimulai, Meksiko mulai. Hanya sembilan menit setelah kick-off, Jiménez berhasil mengakhiri serangan lima pemain dengan tembakan keras dari area penalti. Gol kemenangan Meksiko dicetak pada menit ke-72 setelah tembakan brilian dari kiri oleh Javier Aquino dilanjutkan dengan jebakan yang dengan mudah dikonversi Peralta menjadi gol. Patut dicatat bahwa dalam pertandingan melawan Portugal, Meksiko tertinggal dua kali sebelum akhirnya merebut poin. Di laga ini juga Meksiko harus kebobolan terlebih dahulu untuk akhirnya mencetak gol dan menang. Selain tayangan ulang tersebut, pertandingan tersebut memiliki tiga acara menarik. Pertama, cederanya bek utama Carlos Salcedo. Pemain Eintracht Frankfurt itu diganti pada babak pertama karena cedera bahu. Kedua, pertarungan antara pemain Meksiko dan Selandia Baru setelah pertarungan sengit antara Michael Boxall dan Hector Herrera Terakhir, pidato khusus oleh Rafael Marquez. Marquez yang diperkuat Barcelona digantikan oleh Osorio dan Hector Moreno pada menit ke-68. Moreno sendiri merupakan pemain yang menggantikan Salcedo di babak pertama. Dengan menghadapi Marquez, pemain berusia 38 tahun itu resmi menjadi pemain tertua kedua dalam sejarah Piala Konfederasi. Di pertandingan final, Meksiko akan menghadapi Rusia. Hasil imbang sudah cukup untuk melaju ke semifinal El Tri, terlepas dari hasil yang didapat Portugal dalam pertandingan melawan Selandia Baru. Namun, jika ingin menghindari juara Grup B, tujuan utama Anda adalah menang.'''
c = '''Timnas Indonesia kembali diturunkan setelah pelatih Alfred Riedl memutuskan melepas Rizuki Pera, Septian David Moran, dan Do Mingus Fakdaba. Tiga pemain tersisih setelah timnas Indonesia kembali dari uji coba di Myanmar dan Vietnam. Rizky, David dan Fakdaver jelas tak akan mengikuti pemusatan latihan di Karavaci, Tangerang. “Setelah kembali dari Vietnam, kami melepas Septian David, Lizky Pera yang masuk daftar tunggu, dan Do Mingus.-Catatan: Bagaimana Riedl membentuk Timnas Indonesia Ahli taktik asal Austria itu tidak memberikan detail apapun tentang tindakan disipliner apa yang diambil Fakdaver. Ini sesi latihan nasional di Piala AFF 2016. Riedl memanggil penjaga gawang Semyon Padang Jandia Ekputra, yang sebelumnya dipertahankan untuk pertandingan Tes di Myanmar dan Vietnam.'''
d = '''BALI, CNN Indonesia - Ketua Umum Partai Hanura Usman Sapta Odang mengatakan tidak akan mencampuri pencalonan Joko Widodo di Pilpres 2019. Rapimnas) Hanullah, Bali, Sabtu (5/8). Hanullah belum merumuskan standar atau figur rekan-rekan Jokowi. Menurutnya, hal itu merupakan hak prerogatif mantan Gubernur DKI Jakarta itu. Namun, Osman mencontohkan pasangan Jokowi saat ini, Yusuf Kalla. Meski dengan karakteristik berbeda, Jokowi-JK diyakini mampu menjalankan roda pemerintahan. Jokowi juga diyakini mampu bertahan bersama JK di tengah rumor mesum yang beredar di masyarakat. Nama-nama orang luar juga tidak masuk dalam sistem. Presiden Jokowi dan JK masuk dalam sistem. Sekarang mereka menjalankan pemerintahan dengan baik,"" kata Osman. Ketua Komite Pengurus Hanullah Lapymnath Jafar Bajebel menegaskan bahwa Hanullah tidak akan ikut campur dalam pemilihan wakil presiden. Dia menegaskan, belum ada pembicaraan soal sekutu Jokowi dalam forum-forum panitia. Sebelumnya, pada pembukaan Lapimnas Hanullah yang dihadiri Jokowi, Utsman Sapta mengumumkan akan memenangkan Pilpres DKI 2019. menegaskan kembali posisi partainya di mendukung terpilihnya kembali mantan Gubernur JKI (Mori).'''
e = '''LONDON (ANTARA NEWS) - Umat Kristiani Inggris yang tergabung dalam Persatuan Katolik Kristen Inggris (PERKI) merayakan Natal dengan sate ayam dari Warung Windsor di lobi Queen's Park Community School di London. Deputi Ekonom KBRI London Hastin Dumadi kepada Antara London, Rabu, mengatakan, acara Natal itu didahului kebaktian Natal oleh pendetanya Ray Porter, dilanjutkan Dubes RI untuk Inggris Rizal. oleh Dr. Sukuma bersama istrinya Hana Satriyo. Duta Besar Kristen Inggris Rizal Sukma dan Duta Besar Kana a Satriyo menerima karangan bunga saat memasuki ruang Natal, dan Hana menerima rangkaian bunga untuk tarian dari Ambon. Duta Besar Rizal Sukma memberikan sambutan mengatakan bahwa bangsa Indonesia, terdiri dari perwakilan dari berbagai suku bangsa dan agama, lahir dari keinginan bersama untuk menjadi bangsa yang maju, adil dan makmur. Untuk itu, Dubes Rizal meyakini bahwa dengan menjaga semangat kebangsaan tersebut, Indonesia akan berkembang menjadi negara yang kuat yang melindungi seluruh warga negaranya. Upacara Natal diawali dengan penyalaan lilin oleh Anita Ruflima, DCM KBRI London dan Lenny Feather Patiasena, Ketua PERKI, Kevin Dialdestro, Ketua Panitia Perayaan Natal, dan Samuel, Perwakilan BI di London. . Perayaan Natal diperkaya dengan penampilan grup vokal Kota Bristol dan tarian tradisional oleh grup INDUK, grup vokal anak-anak. Tahun ini, Kevin Dialdestro, seorang mahasiswa Indonesia di Universitas Cambridge, ditunjuk sebagai ketua Panitia Perayaan Natal, dan acara diakhiri dengan makan malam sit windsor, rendang padang, sayuran dan puding.'''

documents = [a, b, c, d, e]
times = []
plagiarism_cases_list = []

for doc in tqdm(documents):
    start = datetime.now()
    plagiarism_cases = detailed_plagiarism(sus_text=doc, n=5, threshold=.8, clustering=False)
    end = datetime.now()
    total_time = end-start
    times.append(total_time)
    plagiarism_cases_list.append(plagiarism_cases)


  0%|          | 0/5 [00:00<?, ?it/s]

[INFO] Transforming suspicious text into IndoBERT embeddings...
[INFO] Getting 5 similar documents by cosine similarity...


100%|██████████| 93870/93870 [00:13<00:00, 7060.32it/s]


[INFO] Getting source documents..
[INFO] Source document 5 appended
[INFO] Source document 22493 appended
[INFO] Source document 41106 appended
[INFO] Source document 60767 appended
[INFO] Source document 75240 appended
[INFO] Performing detailed analysis...
[INFO] Checking source document 5
[INFO] Length of source: 14
[INFO] Length of sus: 14
[INFO] Source Sentence 0
[INFO] Sus Sentence 0
[INFO] Checking current source sentence no 0 with sus sentence no 0...
[INFO] comparing sentences...
[INFO] Sus Sentence 1
[INFO] Checking current source sentence no 0 with sus sentence no 1...
[INFO] comparing sentences...
[INFO] Sus Sentence 2
[INFO] Checking current source sentence no 0 with sus sentence no 2...
[INFO] comparing sentences...
[INFO] Sus Sentence 3
[INFO] Checking current source sentence no 0 with sus sentence no 3...
[INFO] comparing sentences...
[INFO] Sus Sentence 4
[INFO] Checking current source sentence no 0 with sus sentence no 4...
[INFO] comparing sentences...
[INFO] Sus Sen

 20%|██        | 1/5 [00:27<01:51, 27.98s/it]

...

[INFO] Source document 19359 checked
[INFO] Checking source document 29063
[INFO] Source document 29063 checked
[INFO] Checking source document 40165
[INFO] Source document 40165 checked
[INFO] Checking source document 60073
[INFO] Source document 60073 checked
[INFO] Checking source document 73767
[INFO] Source document 73767 checked
[INFO] Transforming suspicious text into IndoBERT embeddings...
[INFO] Getting 5 similar documents by cosine similarity...


100%|██████████| 93870/93870 [00:11<00:00, 8105.18it/s]


[INFO] Getting source documents..
[INFO] Source document 21946 appended
[INFO] Source document 32317 appended
[INFO] Source document 49786 appended
[INFO] Source document 53087 appended
[INFO] Source document 77175 appended
[INFO] Performing detailed analysis...
[INFO] Checking source document 21946
[INFO] Length of source: 8
[INFO] Length of sus: 10
[INFO] Source Sentence 0
[INFO] Sus Sentence 0
[INFO] Checking current source sentence no 0 with sus sentence no 0...
[INFO] comparing sentences...
[INFO] Sus Sentence 1
[INFO] Checking current source sentence no 0 with sus sentence no 1...
[INFO] comparing sentences...
[INFO] Sus Sentence 2
[INFO] Checking current source sentence no 0 with sus sentence no 2...
[INFO] comparing sentences...
[INFO] Sus Sentence 3
[INFO] Checking current source sentence no 0 with sus sentence no 3...
[INFO] comparing sentences...
[INFO] Sus Sentence 4
[INFO] Checking current source sentence no 0 with sus sentence no 4...
[INFO] comparing sentences...
[INFO] 

100%|██████████| 5/5 [02:50<00:00, 34.02s/it]

[INFO] Sus Sentence 9
[INFO] Checking current source sentence no 7 with sus sentence no 9...
[INFO] comparing sentences...
[INFO] Source document 21946 checked
[INFO] Checking source document 32317
[INFO] Source document 32317 checked
[INFO] Checking source document 49786
[INFO] Source document 49786 checked
[INFO] Checking source document 53087
[INFO] Source document 53087 checked
[INFO] Checking source document 77175
[INFO] Source document 77175 checked


In [30]:
times

[datetime.timedelta(seconds=27, microseconds=982435),
 datetime.timedelta(seconds=79, microseconds=867393),
 datetime.timedelta(seconds=16, microseconds=484357),
 datetime.timedelta(seconds=27, microseconds=299342),
 datetime.timedelta(seconds=18, microseconds=470534)]

In [35]:
plagiarism_cases_list[4]

{'overall_plagiarism_score': 0.976452362537384,
 'suspected_sentences': {21946: {'source_doc_url': 'http://www.antaranews.com/berita/601440/sate-ayam-dalam-hidangan-natal-di-inggris',
   'plagiarised_pairs': [{'source_sentence': 'London( ANTARA News)- Umat Kristiani Inggris yang tergabung dalam Persekutuan Umat Kristen dan Katholik Inggris( PERKI) mengelar   peringatan Natal dengan hidangan sate ayam dari Warung Windsor, bertempat di aula Queens Park Community School, London.',
     'input_sentence': "LONDON (ANTARA NEWS) - Umat Kristiani Inggris yang tergabung dalam Persatuan Katolik Kristen Inggris (PERKI) merayakan Natal dengan sate ayam dari Warung Windsor di lobi Queen's Park Community School di London.",
     'similarity_score': 0.9373473525047302},
    {'source_sentence': 'Pelaksana Fungsi Ekonomi KBRI London Hastin Dumadi kepada Antara London, Rabu menyebutkan acara peringatan Natal ini didahului oleh ibadah Natal yang dipimpin pendetanya Ray Porter dan dilanjutkan perayaan Nat

In [26]:
times_clustering = []
plagiarism_cases_list_clustering = []

for doc in tqdm(documents):
    start = datetime.now()
    plagiarism_cases = detailed_plagiarism(sus_text=doc, n=5, threshold=.8, clustering=True)
    end = datetime.now()
    total_time = end-start
    times_clustering.append(total_time)
    plagiarism_cases_list_clustering.append(plagiarism_cases)

  0%|          | 0/5 [00:00<?, ?it/s]

[INFO] Transforming suspicious text into IndoBERT embeddings...
[INFO] Getting 5 similar documents by cosine similarity...
[INFO] Getting source documents..
[INFO] Source document 5 appended
[INFO] Source document 22493 appended
[INFO] Source document 41106 appended
[INFO] Source document 60767 appended
[INFO] Source document 75240 appended
[INFO] Performing detailed analysis...
[INFO] Checking source document 5
[INFO] Length of source: 14
[INFO] Length of sus: 14
[INFO] Source Sentence 0
[INFO] Sus Sentence 0
[INFO] Checking current source sentence no 0 with sus sentence no 0...
[INFO] comparing sentences...
[INFO] Sus Sentence 1
[INFO] Checking current source sentence no 0 with sus sentence no 1...
[INFO] comparing sentences...
[INFO] Sus Sentence 2
[INFO] Checking current source sentence no 0 with sus sentence no 2...
[INFO] comparing sentences...
[INFO] Sus Sentence 3
[INFO] Checking current source sentence no 0 with sus sentence no 3...
[INFO] comparing sentences...
[INFO] Sus Sen

 20%|██        | 1/5 [00:15<01:01, 15.32s/it]

[INFO] Sus Sentence 12
[INFO] Checking current source sentence no 13 with sus sentence no 12...
[INFO] comparing sentences...
[INFO] Sus Sentence 13
[INFO] Checking current source sentence no 13 with sus sentence no 13...
[INFO] comparing sentences...
[INFO] Source document 5 checked
[INFO] Checking source document 22493
[INFO] Source document 22493 checked
[INFO] Checking source document 41106
[INFO] Source document 41106 checked
[INFO] Checking source document 60767
[INFO] Source document 60767 checked
[INFO] Checking source document 75240
[INFO] Source document 75240 checked
[INFO] Transforming suspicious text into IndoBERT embeddings...
[INFO] Getting 5 similar documents by cosine similarity...
[INFO] Getting source documents..
[INFO] Source document 8268 appended
[INFO] Source document 32163 appended
[INFO] Source document 58891 appended
[INFO] Source document 65734 appended
[INFO] Source document 91936 appended
[INFO] Performing detailed analysis...
[INFO] Checking source documen

 40%|████      | 2/5 [01:21<02:15, 45.23s/it]

...

[INFO] Sus Sentence 4
[INFO] Checking current source sentence no 8 with sus sentence no 4...
[INFO] comparing sentences...
[INFO] Sus Sentence 5
[INFO] Checking current source sentence no 8 with sus sentence no 5...
[INFO] comparing sentences...
[INFO] Source document 11613 checked
[INFO] Checking source document 31487
[INFO] Source document 31487 checked
[INFO] Checking source document 44333
[INFO] Source document 44333 checked
[INFO] Checking source document 68629
[INFO] Source document 68629 checked
[INFO] Checking source document 89641
[INFO] Source document 89641 checked
[INFO] Transforming suspicious text into IndoBERT embeddings...
[INFO] Getting 5 similar documents by cosine similarity...
[INFO] Getting source documents..
[INFO] Source document 19359 appended
[INFO] Source document 29063 appended
[INFO] Source document 40165 appended
[INFO] Source document 60073 appended
[INFO] Source document 73767 appended
[INFO] Performing detailed analysis...
[INFO] Checking source document

 80%|████████  | 4/5 [01:39<00:21, 21.44s/it]

[INFO] Source document 19359 checked
[INFO] Checking source document 29063
[INFO] Source document 29063 checked
[INFO] Checking source document 40165
[INFO] Source document 40165 checked
[INFO] Checking source document 60073
[INFO] Source document 60073 checked
[INFO] Checking source document 73767
[INFO] Source document 73767 checked
[INFO] Transforming suspicious text into IndoBERT embeddings...
[INFO] Getting 5 similar documents by cosine similarity...
[INFO] Getting source documents..
[INFO] Source document 21946 appended
[INFO] Source document 32317 appended
[INFO] Source document 49786 appended
[INFO] Source document 53087 appended
[INFO] Source document 77175 appended
[INFO] Performing detailed analysis...
[INFO] Checking source document 21946
[INFO] Length of source: 8
[INFO] Length of sus: 10
[INFO] Source Sentence 0
[INFO] Sus Sentence 0
[INFO] Checking current source sentence no 0 with sus sentence no 0...
[INFO] comparing sentences...
[INFO] Sus Sentence 1
[INFO] Checking c

100%|██████████| 5/5 [01:45<00:00, 21.10s/it]

[INFO] Sus Sentence 8
[INFO] Checking current source sentence no 7 with sus sentence no 8...
[INFO] comparing sentences...
[INFO] Sus Sentence 9
[INFO] Checking current source sentence no 7 with sus sentence no 9...
[INFO] comparing sentences...
[INFO] Source document 21946 checked
[INFO] Checking source document 32317
[INFO] Source document 32317 checked
[INFO] Checking source document 49786
[INFO] Source document 49786 checked
[INFO] Checking source document 53087
[INFO] Source document 53087 checked
[INFO] Checking source document 77175
[INFO] Source document 77175 checked


In [27]:
times_clustering

[datetime.timedelta(seconds=15, microseconds=323644),
 datetime.timedelta(seconds=66, microseconds=164315),
 datetime.timedelta(seconds=3, microseconds=840202),
 datetime.timedelta(seconds=13, microseconds=949075),
 datetime.timedelta(seconds=6, microseconds=233290)]

In [40]:
plagiarism_cases_list_clustering[4]

{'overall_plagiarism_score': 0.976452362537384,
 'suspected_sentences': {21946: {'source_doc_url': 'http://www.antaranews.com/berita/601440/sate-ayam-dalam-hidangan-natal-di-inggris',
   'plagiarised_pairs': [{'source_sentence': 'London( ANTARA News)- Umat Kristiani Inggris yang tergabung dalam Persekutuan Umat Kristen dan Katholik Inggris( PERKI) mengelar   peringatan Natal dengan hidangan sate ayam dari Warung Windsor, bertempat di aula Queens Park Community School, London.',
     'input_sentence': "LONDON (ANTARA NEWS) - Umat Kristiani Inggris yang tergabung dalam Persatuan Katolik Kristen Inggris (PERKI) merayakan Natal dengan sate ayam dari Warung Windsor di lobi Queen's Park Community School di London.",
     'similarity_score': 0.9373473525047302},
    {'source_sentence': 'Pelaksana Fungsi Ekonomi KBRI London Hastin Dumadi kepada Antara London, Rabu menyebutkan acara peringatan Natal ini didahului oleh ibadah Natal yang dipimpin pendetanya Ray Porter dan dilanjutkan perayaan Nat

In [42]:
for i in times:
    print(i)


0:00:27.982435
0:01:19.867393
0:00:16.484357
0:00:27.299342
0:00:18.470534


In [43]:
for i in times_clustering:
    print(i)

0:00:15.323644
0:01:06.164315
0:00:03.840202
0:00:13.949075
0:00:06.233290


In [9]:

# test
document = '''Versi web Google Calendar memiliki sesuatu yang baru untuk ditawarkan. Penampilannya telah sepenuhnya diperbarui, mengadopsi gaya desain modern yang sebelumnya tersedia pada aplikasi Google Calendar di perangkat mobile. Secara keseluruhan, versi Calendar baru terlihat lebih rapi dan lebih mudah untuk diarahkan. Layout responsif yang digunakan berarti penampilannya akan disesuaikan secara otomatis dengan ukuran layar dan jendela browser. Bukan hanya berubah secara visual, versi Calendar baru juga membawa beberapa fitur yang berguna. Salah satunya adalah kemampuan untuk memformat dan menambahkan tautan pada undangan, sehingga pengguna dapat membuat jadwal yang lebih detail dan memberikan akses kepada materi yang dibutuhkan oleh peserta lain sebelum rapat dimulai. Calendar sekarang juga membuat lebih mudah untuk melihat dan mengelola beberapa kalender dari akun yang berbeda dalam tampilan harian. Fitur ini tentunya akan sangat membantu pekerja yang harus menjadwalkan rapat untuk anggota tim mereka. Terakhir, saat membuat jadwal rapat baru, pengguna dapat menambahkan informasi yang lebih detail tentang ruang rapat, termasuk lokasi, kapasitas, peralatan audio dan video, dan informasi tentang aksesibilitas bagi peserta yang berkedudukan kursi roda. Bagi yang menggunakan Calendar untuk akun pribadi mereka, mereka bisa mengaktifkan tampilan baru dengan mengklik tombol "Use new Calendar" di ujung kanan atas. Administrator G Suite juga dapat mengaktifkan tampilan baru Calendar mulai hari ini. Sumber: Google. DailySocial.id adalah portal berita startup dan inovasi teknologi. Kamu bisa menjadi anggota komunitas startup dan inovasi DailySocial.id, mengunduh laporan riset dan statistik seputar teknologi secara gratis, dan mengikuti berita startup dan gadget terbaru di Indonesia.'''
document = '''Meksiko menunjukkan ketahanan lagi. Kamis (22/06), tertinggal dari El Tori berhasil membalikkan keadaan dan akhirnya menjadi 2 - 1 di leg kedua Grup A Piala Konfederasi 2017 di Stadion Olimpiade Fisht Sochi melawan Selandia Baru. Namun, dua gol Raul Jimenez (54') dan Oribe Peralta (72') akhirnya memupus harapan All Whites untuk bangkit di turnamen tersebut. Tambahan tiga poin itu untuk sementara membawa Meksiko ke puncak klasemen Grup A dengan raihan empat poin. Poin mereka sebenarnya sama dengan Portugal, namun tim asuhan Juan Carlos Osorio memiliki kemampuan mencetak gol yang lebih baik. Selandia Baru, sebaliknya, belum mencetak satu poin pun dalam dua pertandingan, sehingga dipastikan akan tersingkir. Meksiko tidak menunjukkan sisi terbaiknya kali ini. Osorio melakukan delapan pergantian pemain dari hasil imbang melawan Portugal. Pemain seperti Marco Fabian, Diego Reyes, Giovani dos Santos dan Oribe Peralta mendapat kesempatan bermain sejak awal. Tak hanya itu, formasi dasar kali ini diubah menjadi 3-4-3, namun sebenarnya perubahan tersebut tidak berdampak besar pada keseluruhan pertandingan melawan Meksiko. Bukti bahwa mereka masih bisa mendominasi lawan, baik dalam penguasaan bola maupun dalam menciptakan peluang. Secara keseluruhan, Meksiko menguasai bola 65% dari waktu. Selain itu, mereka mampu menciptakan 16 peluang mencetak gol, 10 di antaranya tepat sasaran. Tapi mungkin karakter Meksiko tertinggal lebih dulu. Pasalnya, meski selalu dikunci, Selandia Baru justru lebih dulu tampil baik berkat Wood. Pencetak gol terbanyak divisi Championship di Leeds United itu memanfaatkan kekeliruan pertahanan Meksiko, yang mampu menaklukkan kiper Alfredo Tavarel. Begitu babak kedua dimulai, Meksiko mulai. Hanya sembilan menit setelah kick-off, Jiménez berhasil mengakhiri serangan lima pemain dengan tembakan keras dari area penalti. Gol kemenangan Meksiko dicetak pada menit ke-72 setelah tembakan brilian dari kiri oleh Javier Aquino dilanjutkan dengan jebakan yang dengan mudah dikonversi Peralta menjadi gol. Patut dicatat bahwa dalam pertandingan melawan Portugal, Meksiko tertinggal dua kali sebelum akhirnya merebut poin. Di laga ini juga Meksiko harus kebobolan terlebih dahulu untuk akhirnya mencetak gol dan menang. Selain tayangan ulang tersebut, pertandingan tersebut memiliki tiga acara menarik. Pertama, cederanya bek utama Carlos Salcedo. Pemain Eintracht Frankfurt itu diganti pada babak pertama karena cedera bahu. Kedua, pertarungan antara pemain Meksiko dan Selandia Baru setelah pertarungan sengit antara Michael Boxall dan Hector Herrera Terakhir, pidato khusus oleh Rafael Marquez. Marquez yang diperkuat Barcelona digantikan oleh Osorio dan Hector Moreno pada menit ke-68. Moreno sendiri merupakan pemain yang menggantikan Salcedo di babak pertama. Dengan menghadapi Marquez, pemain berusia 38 tahun itu resmi menjadi pemain tertua kedua dalam sejarah Piala Konfederasi. Di pertandingan final, Meksiko akan menghadapi Rusia. Hasil imbang sudah cukup untuk melaju ke semifinal El Tri, terlepas dari hasil yang didapat Portugal dalam pertandingan melawan Selandia Baru. Namun, jika ingin menghindari juara Grup B, tujuan utama Anda adalah menang.'''
document = '''Timnas Indonesia kembali diturunkan setelah pelatih Alfred Riedl memutuskan melepas Rizuki Pera, Septian David Moran, dan Do Mingus Fakdaba. Tiga pemain tersisih setelah timnas Indonesia kembali dari uji coba di Myanmar dan Vietnam. Rizky, David dan Fakdaver jelas tak akan mengikuti pemusatan latihan di Karavaci, Tangerang. “Setelah kembali dari Vietnam, kami melepas Septian David, Lizky Pera yang masuk daftar tunggu, dan Do Mingus.-Catatan: Bagaimana Riedl membentuk Timnas Indonesia Ahli taktik asal Austria itu tidak memberikan detail apapun tentang tindakan disipliner apa yang diambil Fakdaver. Ini sesi latihan nasional di Piala AFF 2016. Riedl memanggil penjaga gawang Semyon Padang Jandia Ekputra, yang sebelumnya dipertahankan untuk pertandingan Tes di Myanmar dan Vietnam.'''
document = '''BALI, CNN Indonesia - Ketua Umum Partai Hanura Usman Sapta Odang mengatakan tidak akan mencampuri pencalonan Joko Widodo di Pilpres 2019. Rapimnas) Hanullah, Bali, Sabtu (5/8). Hanullah belum merumuskan standar atau figur rekan-rekan Jokowi. Menurutnya, hal itu merupakan hak prerogatif mantan Gubernur DKI Jakarta itu. Namun, Osman mencontohkan pasangan Jokowi saat ini, Yusuf Kalla. Meski dengan karakteristik berbeda, Jokowi-JK diyakini mampu menjalankan roda pemerintahan. Jokowi juga diyakini mampu bertahan bersama JK di tengah rumor mesum yang beredar di masyarakat. Nama-nama orang luar juga tidak masuk dalam sistem. Presiden Jokowi dan JK masuk dalam sistem. Sekarang mereka menjalankan pemerintahan dengan baik,"" kata Osman. Ketua Komite Pengurus Hanullah Lapymnath Jafar Bajebel menegaskan bahwa Hanullah tidak akan ikut campur dalam pemilihan wakil presiden. Dia menegaskan, belum ada pembicaraan soal sekutu Jokowi dalam forum-forum panitia. Sebelumnya, pada pembukaan Lapimnas Hanullah yang dihadiri Jokowi, Utsman Sapta mengumumkan akan memenangkan Pilpres DKI 2019. menegaskan kembali posisi partainya di mendukung terpilihnya kembali mantan Gubernur JKI (Mori).'''
document = '''LONDON (ANTARA NEWS) - Umat Kristiani Inggris yang tergabung dalam Persatuan Katolik Kristen Inggris (PERKI) merayakan Natal dengan sate ayam dari Warung Windsor di lobi Queen's Park Community School di London. Deputi Ekonom KBRI London Hastin Dumadi kepada Antara London, Rabu, mengatakan, acara Natal itu didahului kebaktian Natal oleh pendetanya Ray Porter, dilanjutkan Dubes RI untuk Inggris Rizal. oleh Dr. Sukuma bersama istrinya Hana Satriyo. Duta Besar Kristen Inggris Rizal Sukma dan Duta Besar Kana a Satriyo menerima karangan bunga saat memasuki ruang Natal, dan Hana menerima rangkaian bunga untuk tarian dari Ambon. Duta Besar Rizal Sukma memberikan sambutan mengatakan bahwa bangsa Indonesia, terdiri dari perwakilan dari berbagai suku bangsa dan agama, lahir dari keinginan bersama untuk menjadi bangsa yang maju, adil dan makmur. Untuk itu, Dubes Rizal meyakini bahwa dengan menjaga semangat kebangsaan tersebut, Indonesia akan berkembang menjadi negara yang kuat yang melindungi seluruh warga negaranya. Upacara Natal diawali dengan penyalaan lilin oleh Anita Ruflima, DCM KBRI London dan Lenny Feather Patiasena, Ketua PERKI, Kevin Dialdestro, Ketua Panitia Perayaan Natal, dan Samuel, Perwakilan BI di London. . Perayaan Natal diperkaya dengan penampilan grup vokal Kota Bristol dan tarian tradisional oleh grup INDUK, grup vokal anak-anak. Tahun ini, Kevin Dialdestro, seorang mahasiswa Indonesia di Universitas Cambridge, ditunjuk sebagai ketua Panitia Perayaan Natal, dan acara diakhiri dengan makan malam sit windsor, rendang padang, sayuran dan puding.'''
# sentences = sent_tokenize(document)
# print(sentences)
start = datetime.now()
plagiarism_cases = detailed_plagiarism(sus_text=document, n=5, threshold=.5, clustering=False)
print(plagiarism_cases)
end = datetime.now()
total_time = end-start
print(f"total time: {total_time}")

[INFO] Transforming suspicious text into IndoBERT embeddings...
[INFO] Getting 5 similar documents by cosine similarity...


100%|██████████| 93870/93870 [00:10<00:00, 8972.88it/s]


[INFO] Getting source documents..
[INFO] Source document 8268 appended
[INFO] Source document 32163 appended
[INFO] Source document 58891 appended
[INFO] Source document 65734 appended
[INFO] Source document 91936 appended
[INFO] Performing detailed analysis...
[INFO] Checking source document 8268
[INFO] Length of source: 33
[INFO] Length of sus: 31
[INFO] Source Sentence 0
[INFO] Sus Sentence 0
[INFO] Checking current source sentence no 0 with sus sentence no 0...
[INFO] comparing sentences...
[INFO] Sus Sentence 1
[INFO] Checking current source sentence no 0 with sus sentence no 1...
[INFO] comparing sentences...
[INFO] Sus Sentence 2
[INFO] Checking current source sentence no 0 with sus sentence no 2...
[INFO] comparing sentences...
[INFO] Sus Sentence 3
[INFO] Checking current source sentence no 0 with sus sentence no 3...
[INFO] comparing sentences...
[INFO] Sus Sentence 4
[INFO] Checking current source sentence no 0 with sus sentence no 4...
[INFO] comparing sentences...
[INFO] S

In [10]:
plagiarism_cases['suspected_sentences'][8268]['plagiarised_pairs']

[{'source_sentence': 'Meksiko kembali menunjukkan daya tahannya.',
  'input_sentence': 'Meksiko menunjukkan ketahanan lagi.',
  'similarity_score': 0.8348720669746399},
 {'source_sentence': 'Meksiko kembali menunjukkan daya tahannya.',
  'input_sentence': 'Poin mereka sebenarnya sama dengan Portugal, namun tim asuhan Juan Carlos Osorio memiliki kemampuan mencetak gol yang lebih baik.',
  'similarity_score': 0.5132771730422974},
 {'source_sentence': 'Meksiko kembali menunjukkan daya tahannya.',
  'input_sentence': 'Meksiko tidak menunjukkan sisi terbaiknya kali ini.',
  'similarity_score': 0.7778301239013672},
 {'source_sentence': 'Meksiko kembali menunjukkan daya tahannya.',
  'input_sentence': 'Osorio melakukan delapan pergantian pemain dari hasil imbang melawan Portugal.',
  'similarity_score': 0.5090059041976929},
 {'source_sentence': 'Meksiko kembali menunjukkan daya tahannya.',
  'input_sentence': 'Bukti bahwa mereka masih bisa mendominasi lawan, baik dalam penguasaan bola maupun 

In [8]:
with open('data/df.pkl', 'rb') as f:
    loaded = load(f)

In [9]:
loaded['paragraphs_sent_tok'][89641]

['Jumlah skuat tim nasional Indonesia kembali mengalami pengurangan setelah pelatih Alfred Riedl memutuskan untuk melepas Rizky Pellu, Septian David Maulana dan Dominggus Fakdawer.',
 'Ketiga pemain tersebut dicoret setelah timnas Indonesia pulang dari laga uji coba di Myanmar dan Vietnam.',
 'Rizky, David dan Fakdawer dipastikan takkan ikut serta dalam pemusatan latihan di Karawaci, Tangerang."',
 'Setelah pulang dari Vietnam kami melepas Septian David, Rizky Pellu yang masuk waiting list \xa0 dan Dominggus.',
 'Dominggus harus kami coret karena indisipliner," kata Riedl usai memimpin sesi latihan, Jumat( 11/ 11) pagi.',
 'PIALA AFF- CATATAN: Bagaimana Riedl Membentuk Timnas Indonesia Juru taktik asal Austria itu tak memberikan detail soal tindakkan indisipliner apa yang dilakukan oleh Fakdawer.',
 'Pemain milik Persipura Jayapura itu pun belum sekali pun dimainkan Riedl dalam laga uji coba timnas Indonesia.',
 'Pada pelatnas ini, terdapat 24 pemain proyeksi timnas Indonesia untuk Pia

In [10]:
plagiarism_cases['suspected_sentences'][7864]['plagiarised_pairs']

[{'source_sentence': 'Perkembangan pesat teknologi augmented reality melahirkan kategori baru permainan hybrid yang melebur dunia fisik dan digital.',
  'input_sentence': 'Versi web Google Calendar memiliki sesuatu yang baru untuk ditawarkan.',
  'similarity_score': 0.9993436336517334},
 {'source_sentence': 'Perkembangan pesat teknologi augmented reality melahirkan kategori baru permainan hybrid yang melebur dunia fisik dan digital.',
  'input_sentence': 'Penampilannya telah sepenuhnya diperbarui, mengadopsi gaya desain modern yang sebelumnya tersedia pada aplikasi Google Calendar di perangkat mobile.',
  'similarity_score': 0.9995989203453064},
 {'source_sentence': 'Perkembangan pesat teknologi augmented reality melahirkan kategori baru permainan hybrid yang melebur dunia fisik dan digital.',
  'input_sentence': 'Secara keseluruhan, versi Calendar baru terlihat lebih rapi dan lebih mudah untuk diarahkan.',
  'similarity_score': 0.9997172355651855},
 {'source_sentence': 'Perkembangan p